In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages tos load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import shutil
import cv2
import os
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Activation, GlobalAveragePooling2D, Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import Input

%matplotlib inline
plt.rcParams["figure.figsize"] = (17, 6) # (w, h)

In [ ]:
TRAINING_DIR = "../input/cassava-leaf-disease-classification/train_images"
TRAINING_CSV = "../input/cassava-leaf-disease-classification/train.csv"
JSON_LABELS = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
#PRETRAINED_MODEL = "../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
train_df = pd.read_csv(TRAINING_CSV)
train_df["label"] = train_df["label"].astype("string") # for Keras flow_from_dataframe

In [ ]:
total_images_count = len(train_df.index)
total_train_img_count = int(len(train_df.index) * 0.8)
total_val_img_count = total_images_count - total_train_img_count
print("Expected images coutns:")
print("\nTotal Images from original directory: {}".format(total_images_count))
print("Training Images: {}".format(total_train_img_count))
print("Validation Images: {}".format(total_val_img_count))

In [ ]:
label_df = pd.read_json(JSON_LABELS, orient = 'index')
label_df = label_df.values.flatten().tolist()
label_df

In [ ]:
train_label_0 = train_df[train_df["label"]== "0"]
train_label_1 = train_df[train_df["label"]== "1"]
train_label_2 = train_df[train_df["label"]== "2"]
train_label_3 = train_df[train_df["label"]== "3"]
train_label_4 = train_df[train_df["label"]== "4"]
len(train_label_4)

In [ ]:
training_images_dir = TRAINING_DIR + "/*.jpg"
print(training_images_dir)
training_images = glob.glob(training_images_dir)
plt.figure(figsize=(12, 12))    
for i in range(1, 10):
    training_image = np.random.choice(training_images)
    training_image_RGB = cv2.imread(training_image)[...,::-1]
    print(training_image_RGB.shape)
    plt.subplot(3, 3, i)
    plt.imshow(training_image_RGB)
    plt.axis('off')

In [ ]:
training_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 100,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.3,
    brightness_range = [0.7, 1.4],
    horizontal_flip = True,
    vertical_flip=True,
    fill_mode = "nearest",
    validation_split=0.2
)

validation_datagen = ImageDataGenerator(
    rescale = 1/255,
    validation_split=0.2
)

In [ ]:
BATCH_SIZE = 24
IMG_WIDTH = 300
IMG_HEIGHT = 300
CHANNEL = 3

print("\nTraining Dataset")
train_ds = training_datagen.flow_from_dataframe(
    train_df,
    TRAINING_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    x_col = "image_id",
    y_col = "label",
    shuffle = True,
    subset = "training"

)
print("\nValidation Dataset")
validation_ds = validation_datagen.flow_from_dataframe(
    train_df,
    TRAINING_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    x_col = "image_id",
    y_col = "label",
    shuffle = False,
    subset = "validation"
)
print("\nClass Indices:")
print(train_ds.class_indices)

In [ ]:
class theCallBacks(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if((logs.get("val_accuracy")>0.92) and (logs.get("accuracy")>0.92)): 
            print("\Training Accuracy> 0.92 & Validation Accuracy> 0.92\nCancelling training!")
            self.model.stop_training = True

            
callback_on_metrics = theCallBacks() #Instantiate theCallBacks

reduce_lr = ReduceLROnPlateau(
                    monitor='val_loss', 
                    factor=0.5,
                    patience= 2, 
                    verbose = 1,
                    cooldown = 1,
                    min_lr=0.0001)

In [ ]:
loss_func = CategoricalCrossentropy()
#optimizer = Adam(learning_rate=0.001)

In [ ]:
import datetime
class LearningRateLogger(Callback):
     def __init__(self):
         super().__init__()
         self._supports_tf_logs = True

     def on_epoch_end(self, epoch, logs=None):
         if logs is None or "learning_rate" in logs:
             return
         logs["learning_rate"] = self.model.optimizer.lr
        
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)     


In [ ]:
new_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNEL))

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
!pip install keras_applications

In [ ]:
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from tensorflow.keras.layers import Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
from keras.engine import Model

In [ ]:
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 201
base_model = VGGFace(model='senet50', include_top=False,input_tensor=new_input, pooling='avg')
last_layer = base_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc7')(x)
x = Dropout(DROPOUT_RATE)(x)
# l=0
# for layer in vgg_notop.layers:
#     print(layer,"["+str(l)+"]")
#     l=l+1

batch_norm_indices = [2, 6, 9, 12, 21, 25, 28, 31, 42, 45, 48, 59, 62, 65, 74, 78, 81, 84, 95, 98, 101, 112, 115, 118, 129, 132, 135, 144, 148, 151, 154, 165, 168, 171, 182, 185, 188, 199, 202, 205, 216, 219, 222, 233, 236, 239, 248, 252, 255, 258, 269, 272, 275]
for i in range(FROZEN_LAYER_NUM):
    if i not in batch_norm_indices:
        base_model.layers[i].trainable = False
# print('vgg layer 2 is trainable: ' + str(vgg_notop.layers[2].trainable))
# print('vgg layer 3 is trainable: ' + str(vgg_notop.layers[3].trainable))

out = Dense(5, activation='softmax', name='classifier')(x)

model = Model(base_model.input, out)

SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001
#optim = keras.optimizers.Adam(lr=ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optim = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = keras.optimizers.SGD(lr=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
#rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#Name = "tf-SeNet50cs230{}".format(int(time.time()))
#tensorboard= TensorBoard(log_dir='logs/{}'.format(Name))
# plot_model(model, to_file='model2.png', show_shapes=True)

In [ ]:
num_epochs = 10
steps_per_epoch = total_train_img_count // BATCH_SIZE

In [ ]:
model_checkpoint_path="../input/cassava-model"

checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True,mode='max')

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("../input/cassava-model/cassava_Model_Senet.h5")
predicted = []
sample_submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in sample_submission.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = tf.reshape(img, (-1, IMG_WIDTH, IMG_HEIGHT, CHANNEL))
    prediction = model.predict(img/255)  
    predicted.append(np.argmax(prediction))

submission = pd.DataFrame({'image_id': sample_submission.image_id, 'label': predicted})
submission.to_csv('submission.csv', index=False) 